In [12]:
import tensorflow as tf
import tensorflow.keras as keras
import random
import os

In [13]:
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
  for gpu in gpus:
    try:
      print("Enabling GPU: ", gpu)
      tf.config.experimental.set_visible_devices(gpu, 'GPU')
      tf.config.experimental.set_memory_growth(gpu, True)
    except Exception as e:
      print(e)

Enabling GPU:  PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [14]:
val_size = 0.3
rand_seed = random.randint(0, 2**32 - 1)
img_dim = 96
batch_size = 8

In [15]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  os.path.join("data", "training"),
  validation_split = val_size,
  subset = "training",
  seed = rand_seed,
  image_size = (img_dim, img_dim),
  batch_size = batch_size,
  shuffle = True)

val_ds = tf.keras.utils.image_dataset_from_directory(
  os.path.join("data", "training"),
  validation_split = val_size,
  subset = "validation",
  seed = rand_seed,
  image_size = (img_dim, img_dim),
  batch_size = batch_size,
  shuffle = True)

Found 1970 files belonging to 2 classes.
Using 1379 files for training.
Found 1970 files belonging to 2 classes.
Using 591 files for validation.


In [16]:
image_input = keras.Input(shape = (img_dim, img_dim, 3))
x = keras.layers.Rescaling(1.0 / 255)(image_input)
x = keras.layers.Conv2D(128, kernel_size = (3, 3), activation = 'relu')(x)
x = keras.layers.MaxPooling2D(pool_size = (2, 2))(x)
x = keras.layers.Conv2D(256, kernel_size = (3, 3), activation = 'relu')(x)
x = keras.layers.MaxPooling2D(pool_size = (2, 2))(x)
x = keras.layers.Dropout(0.05)(x)
feature_output = keras.layers.Flatten()(x)
x = keras.layers.Dense(2048, activation = 'relu')(feature_output)
classification_output = keras.layers.Dense(2)(x)

model = keras.models.Model(image_input, classification_output)

In [17]:
opt = keras.optimizers.Adagrad(learning_rate = 0.01)
model.compile(optimizer = opt,
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
              metrics=['accuracy'])
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 96, 96, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_1 (Rescaling)         │ (None, 96, 96, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 94, 94, 128)    │         3,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 47, 47, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 45, 45, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 22, 22, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 22, 22, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 123904)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2048)           │   253,757,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │         4,098 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 254,060,290 (969.16 MB)

 Trainable params: 254,060,290 (969.16 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
class_names = train_ds.class_names
AUTOTUNE = tf.data.AUTOTUNE
epochs = 16

In [19]:
hist = model.fit(
  train_ds,
  validation_data = val_ds,
  epochs = epochs, 
  callbacks = []
)

Epoch 1/16
173/173 ━━━━━━━━━━━━━━━━━━━━ 14s 72ms/step - accuracy: 0.6282 - loss: 0.6712 - val_accuracy: 0.8697 - val_loss: 0.3430
Epoch 2/16
173/173 ━━━━━━━━━━━━━━━━━━━━ 12s 72ms/step - accuracy: 0.9203 - loss: 0.2196 - val_accuracy: 0.9425 - val_loss: 0.1769
Epoch 3/16
173/173 ━━━━━━━━━━━━━━━━━━━━ 13s 72ms/step - accuracy: 0.9570 - loss: 0.1132 - val_accuracy: 0.9594 - val_loss: 0.1251
Epoch 4/16
173/173 ━━━━━━━━━━━━━━━━━━━━ 13s 72ms/step - accuracy: 0.9812 - loss: 0.0620 - val_accuracy: 0.9628 - val_loss: 0.1051
Epoch 5/16
173/173 ━━━━━━━━━━━━━━━━━━━━ 13s 72ms/step - accuracy: 0.9819 - loss: 0.0479 - val_accuracy: 0.9814 - val_loss: 0.0709
Epoch 6/16
173/173 ━━━━━━━━━━━━━━━━━━━━ 13s 73ms/step - accuracy: 0.9835 - loss: 0.0369 - val_accuracy: 0.9763 - val_loss: 0.0783
Epoch 7/16
173/173 ━━━━━━━━━━━━━━━━━━━━ 13s 73ms/step - accuracy: 0.9934 - loss: 0.0252 - val_accuracy: 0.9831 - val_loss: 0.0666
Epoch 8/16
173/173 ━━━━━━━━━━━━━━━━━━━━ 13s 73ms/step - accuracy: 0.9944 - loss: 0.0210 - 

In [20]:
test_ds = tf.keras.utils.image_dataset_from_directory(
  os.path.join("data", "test"),
  shuffle = False,
  image_size = (img_dim, img_dim)
)

Found 30 files belonging to 2 classes.


In [21]:
model.evaluate(test_ds)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 1.0000 - loss: 0.0284


[0.02839045785367489, 1.0]